In [1]:
import tkinter as tk
from customtkinter import *
import pandas as pd
import SupportFunctions
import threading
import queue
from datetime import datetime
import os
import re
import logging
from collections import defaultdict
import glob


logger_filepath = SupportFunctions.get_data_file_path(filename='scrape_app.log', folder='database\logs')
logging.basicConfig(filename=logger_filepath, level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Get the directory path of main.py
current_dir = r"D:\Users\Arian\Documents\Kubrick\Projects\Internal MI Copmetitor Profiling\save_data\MI_Simon"
metadata_filepath = SupportFunctions.get_data_file_path(filename = 'company_metadata.csv')

# Read in metadata file
company_df = pd.read_csv(filepath_or_buffer=metadata_filepath, index_col = 0)
company_df.fillna('',inplace=True)
# Filter to exclude not yet completed companies
mask = company_df['scraper'] != 'to complete'

df_full_company_list = company_df[['company_name','ticker']].fillna('')
company_df = company_df[mask]
full_company_list = dict(zip(df_full_company_list['company_name'], df_full_company_list['ticker']))

In [2]:
import scrapers

df1 = scrapers.scraper_epam()

In [9]:
# company_name = 'EPAM Systems Inc'
# company_url, scraper, status, ticker = SupportFunctions.get_company_metadata(company_name, company_df)
# sanitized_name = re.sub(r'[<>:"/\\|?*]', '_', company_name).strip()
# file_path = SupportFunctions.get_data_file_path(f'*webscrape*.csv', f'database\output\{sanitized_name}')
# file_path = glob.glob(file_path)[0]
# midiff_file_path = SupportFunctions.get_data_file_path(f'{sanitized_name} diff.csv', f'database\output\{sanitized_name}')
# template_file_path = SupportFunctions.get_data_file_path('Template.csv')
# yahoo_json = SupportFunctions.get_company_details2(status, ticker)
# final_df1 = SupportFunctions.create_final_df2(company_name, company_url, status, yahoo_json, df1, template_file_path)
# # final_df1.loc[0, 'Practices'] = 'Testing'
# old_df = SupportFunctions.read_from_csv(file_path, company_name, template_file_path)
# SupportFunctions.log_new_and_modified_rows3(final_df1, old_df, midiff_file_path, company_name)
# SupportFunctions.write_to_csv(final_df1, file_path, sanitized_name)



company_name = 'EPAM Systems Inc'
company_url, scraper, status, ticker = SupportFunctions.get_company_metadata(company_name, company_df)
sanitized_name = re.sub(r'[<>:"/\\|?*]', '_', company_name).strip()
file_path = SupportFunctions.get_data_file_path(f'*webscrape*.csv', f'database\output\{sanitized_name}')
file_path = glob.glob(file_path)[0]
midiff_file_path = SupportFunctions.get_data_file_path(f'{sanitized_name} diff.csv', f'database\output\{sanitized_name}')
template_file_path = SupportFunctions.get_data_file_path('Template.csv')
yahoo_json = SupportFunctions.get_company_details2(status, ticker)
final_df1 = SupportFunctions.create_final_df2(company_name, company_url, status, yahoo_json, df1, template_file_path)
# final_df1.loc[0, 'Practices'] = 'Testing'
old_df = SupportFunctions.read_from_csv(file_path, company_name, template_file_path)
SupportFunctions.reallocate_old_df(old_df, file_path, sanitized_name)
SupportFunctions.log_new_and_modified_rows3(final_df1, old_df, midiff_file_path, company_name)
SupportFunctions.write_to_csv(final_df1, file_path, sanitized_name)

Removed directory: d:\Users\Arian\Documents\Kubrick\Projects\Internal MI Copmetitor Profiling\save_data\MI_Simon\database\output\EPAM Systems Inc\EPAM Systems Inc-webscrape-June-2024.csv
d:\Users\Arian\Documents\Kubrick\Projects\Internal MI Copmetitor Profiling\save_data\MI_Simon\database\output\EPAM Systems Inc\EPAM Systems Inc-webscrape-June-2024.csv


In [ ]:

def log_differences(message, company_name, count):
    # Dummy function to simulate logging
    print(f"{company_name}: {message} - {count} rows")
def abc(final_df, old_df, path_file, company_name):
    """
    Compare rows between final_df and old_df, update or remove rows accordingly, 
    and handle Excel file updates.

    Args:
    final_df (pd.DataFrame): The most up-to-date dataframe.
    old_df (pd.DataFrame): The older dataframe to compare against.
    path_file (str): Path to save the CSV file.
    company_name (str): Name of the company for logging purposes.
    """
    # Subset to relevant columns
    columns_of_interest = ['Practices', 'Practices_URL', 'Services', 'Services_URL', 'Solutions', 'Solutions_URL']
    df1 = final_df[columns_of_interest].copy()
    df2 = old_df[columns_of_interest].copy()

    # Convert columns to the same data type (string) for comparison
    for column in columns_of_interest:
        df1[column] = df1[column].astype(str)
        df2[column] = df2[column].astype(str)

    # Initialize the temporary DataFrame to store unmatched rows with their status
    temp_df = pd.DataFrame(columns=['Status'] + list(final_df.columns))

    # Keep track of original indices before dropping rows
    original_idx1 = df1.index.tolist()
    original_idx2 = df2.index.tolist()

    # Compare each row in df1 with each row in df2
    for idx1, row1 in df1.iterrows():
        matched = False
        for idx2, row2 in df2.iterrows():
            if row1.equals(row2):
                # If a match is found, remove the row from both dataframes
                df1.drop(idx1, inplace=True)
                df2.drop(idx2, inplace=True)
                matched = True
                break
        if not matched:
            # If no match is found, mark the row as 'Added' and store in temp_df
            full_row1 = final_df.loc[original_idx1[idx1]].copy()
            full_row1['Status'] = 'Added'
            temp_df = pd.concat([temp_df, pd.DataFrame([full_row1])], ignore_index=True)

    # Any remaining rows in df2 are 'Removed'
    for idx2, row2 in df2.iterrows():
        full_row2 = old_df.loc[original_idx2[idx2]].copy()
        full_row2['Status'] = 'Removed'
        temp_df = pd.concat([temp_df, pd.DataFrame([full_row2])], ignore_index=True)

    # If there are any changes, write them to the CSV file
    if not temp_df.empty:
        temp_df.to_csv(path_file, index=False)
        log_differences("Changed", company_name, len(temp_df))
    else:
        # If there are no new or modified rows, delete the CSV file if it exists
        if os.path.exists(path_file):
            os.remove(path_file)
        log_differences("No differences found", company_name, 0)
abc(final_df1, old_df, midiff_file_path, company_name)

In [7]:
def reallocate_olddf(old_df, file_path, sanitized_name):
    history_file_path = SupportFunctions.get_data_file_path(os.path.basename(file_path), f'database\history\{sanitized_name}')
    # Remove the directory containing the csv_file
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f'Removed directory: {file_path}')
    else:
        print(f'Directory does not exist: {file_path}')
    old_df.to_csv(history_file_path, index=False)




In [6]:
import os
import pandas as pd

def collect_diff_csv_files(main_directory):
    # Initialize an empty dataframe
    main_df = pd.DataFrame()
    
    # Walk through each directory and subdirectory
    for root, dirs, files in os.walk(main_directory):
        for file in files:
            # Check if the file is a csv and contains 'diff' in its name
            if file.endswith('.csv') and 'diff' in file:
                file_path = os.path.join(root, file)
                # Read the CSV file and append it to the main dataframe
                temp_df = pd.read_csv(file_path)
                main_df = pd.concat([main_df, temp_df], ignore_index=True)
    
    # If the main dataframe is not empty, save it to the original path directory
    if not main_df.empty:
        output_file = os.path.join(main_directory, f'summary_diff_files_.csv')
        main_df.to_csv(output_file, index=False)
        print(f'Merged dataframe saved to {output_file}')
    else:
        print('No diff CSV files found.')

collect_diff_csv_files(r'D:\Users\Arian\Documents\Kubrick\Projects\Internal MI Copmetitor Profiling\save_data\MI_Simon\database\output')

Merged dataframe saved to D:\Users\Arian\Documents\Kubrick\Projects\Internal MI Copmetitor Profiling\save_data\MI_Simon\database\output\summary_diff_files.csv


In [7]:
import gc

gc.collect()

0